In [1]:
%env THEANO_FLAGS=device=gpu6, lib.cnmem=0.95

from IPython.display import clear_output

import theano
import theano.tensor as T

import lasagne

import os
from preproc import pics2array

import matplotlib.pyplot as plt
%matplotlib inline

env: THEANO_FLAGS=device=gpu6, lib.cnmem=0.95


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 6: GeForce GTX 1080 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)


In [2]:
from lasagne.layers import get_output, get_all_layers, get_all_params

from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import Deconv2DLayer as DeconvLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import DenseLayer, InputLayer

from lasagne.updates import adam

from lasagne.nonlinearities  import LeakyRectify, sigmoid
lref = LeakyRectify(0.2)

In [3]:
def buildGenerator(input_shape, input_var=None, n_base_filter=32):
    net = {}
    
    net["input"] = InputLayer(input_shape, input_var)
    
    net["downs_1"] = ConvLayer(net["input"], n_base_filter, 4)
    net["downs_2"] = ConvLayer(net["downs_1"], n_base_filter*2, 4)
    net["downs_3"] = ConvLayer(net["downs_2"], n_base_filter*4, 4)
    net["downs_4"] = ConvLayer(net["downs_3"], n_base_filter*8, 4)
    
    net["upsc_1"] = DeconvLayer(net["downs_4"], n_base_filter*8, 4)
    net["upsc_2"] = DeconvLayer(net["upsc_1"], n_base_filter*4, 4)
    net["upsc_3"] = DeconvLayer(net["upsc_2"], n_base_filter*2, 4)
    net["output"] = DeconvLayer(net["upsc_3"], input_shape[1], 4, nonlinearity=None)
    
    return net

In [4]:
def buildDiscriminator(input_shape, input_var=None, n_base_filter=32):
    net = {}
    
    net["input"] = InputLayer(input_shape, input_var)
    
    net["downs_1"] = ConvLayer(net["input"], n_base_filter, 4, nonlinearity=lref)
    net["downs_2"] = ConvLayer(net["downs_1"], n_base_filter*2, 4, nonlinearity=lref)
    net["downs_3"] = ConvLayer(net["downs_2"], n_base_filter*4, 4, nonlinearity=lref)
    net["downs_4"] = ConvLayer(net["downs_3"], n_base_filter*8, 4, nonlinearity=lref)
    
    net["dense_1"] = DenseLayer(net["downs_4"], 256, nonlinearity=lref)
    net["output"] = DenseLayer(net["dense_1"], 1, nonlinearity=sigmoid)
    
    return net

In [5]:
PIC_DIMS = [None, 3, 64, 64]

inputA = T.tensor4("Input of Domain A")
inputB = T.tensor4("Input of Domain B")

generatorAB = buildGenerator(PIC_DIMS, inputA)
generatorBA = buildGenerator(PIC_DIMS, inputB)

discriminatorA = buildDiscriminator(PIC_DIMS)
discriminatorB = buildDiscriminator(PIC_DIMS)

In [6]:
outputAB = get_output(generatorAB["output"], {generatorAB["input"] : inputA})
outputBA = get_output(generatorBA["output"], {generatorBA["input"] : inputB})

outputABA = get_output(generatorBA["output"], {generatorBA["input"] : outputAB})
outputBAB = get_output(generatorAB["output"], {generatorAB["input"] : outputBA})

fake_output_prob_a = get_output(discriminatorA["output"], {discriminatorA["input"] : outputBA})
legit_output_prob_a = get_output(discriminatorA["output"], {discriminatorA["input"] : inputA})

fake_output_prob_b = get_output(discriminatorB["output"], {discriminatorB["input"] : outputAB})
legit_output_prob_b = get_output(discriminatorB["output"], {discriminatorB["input"] : inputB})

In [7]:
GA_params = get_all_params(generatorAB["output"], trainable=True)
GB_params = get_all_params(generatorBA["output"], trainable=True)
DA_params = get_all_params(discriminatorA["output"], trainable=True)
DB_params = get_all_params(discriminatorB["output"], trainable=True)

In [8]:
LAMBDA = 100

A_l1 = T.abs_(inputA - outputABA).mean()
B_l1 = T.abs_(inputB - outputBAB).mean()

A_discr = -(T.log(fake_output_prob_a)).mean()
B_discr = -(T.log(fake_output_prob_b)).mean()

genA_loss = A_l1 * LAMBDA + A_discr
genB_loss = B_l1 * LAMBDA + B_discr

discrA_loss = -(T.log(legit_output_prob_a) + T.log(1 - fake_output_prob_a)).mean()
discrB_loss = -(T.log(legit_output_prob_b) + T.log(1 - fake_output_prob_b)).mean()

In [9]:
ADAM_PARAMS = {"learning_rate" : 2e-4, "beta1" : 0.5}

updates = adam(genA_loss, GA_params, **ADAM_PARAMS)
updates.update(adam(genB_loss, GB_params, **ADAM_PARAMS))
updates.update(adam(discrA_loss, DA_params, **ADAM_PARAMS))
updates.update(adam(discrB_loss, DB_params, **ADAM_PARAMS))

In [10]:
train_fun = theano.function([inputA, inputB], [A_l1, A_discr, B_l1, B_discr, discrA_loss, discrB_loss],
                           updates=updates, allow_input_downcast=True)

In [ ]:
#load data

x = [pic for pic in os.listdir("data/") if pic.split(".")[-1] == "jpg"]
y = [pic for pic in os.listdir("data/") if pic.split(".")[-1] == "png"]

x = pics2array("data/", sorted(x), PIC_DIMS[2:])
y = pics2array("data/", sorted(y), PIC_DIMS[2:])

In [ ]:
N_EPOCHS = 200
BATCH_SIZE = 1

for epoch in range(N_EPOCHS):
    clear_output()
    print(epoch)
    
    for i in range(0, len(x) - BATCH_SIZE, BATCH_SIZE):
        train_fun(x[i : i + BATCH_SIZE], y[i : i + BATCH_SIZE])

0
